This notebook evaluates the results of my approach in the vision room 1 easy.

In [1]:
import pandas as pd
import json
import numpy as np
import plotly.graph_objects as go
from pathlib import Path

from evaluation_utils import (
    read_ground_truth_trajectory,
    read_ply,
    read_trajectory_dump,
    get_random_values,
    get_cam_data,
)


In [2]:
initial_keyframe_index_1 = 100
initial_keyframe_index_2 = 120

gt_trajectory, initial_poses = read_ground_truth_trajectory(
    "data/euroc_mav/V1_01_easy/cam0/sensor.yaml",
    "data/euroc_mav/V1_01_easy/state_groundtruth_estimate0/data.csv",
    "data/euroc_mav/V1_01_easy/cam0/data.csv",
    [initial_keyframe_index_1, initial_keyframe_index_2],
)

gt_pcl = read_ply("data/euroc_mav/V1_01_easy/pointcloud0/data.ply")
(min_x, min_y, min_z), (max_x, max_y, max_z) = np.quantile(
    gt_pcl, [0.0001, 0.9999], axis=0
)

(width, height), (fx, fy, cx, cy), (k1, k2, p1, p2) = get_cam_data(
    "data/euroc_mav/V1_01_easy/cam0/sensor.yaml"
)

random_config = {
    "verbose": False,
    "visual": False,
    "do_dump_map": False,
    "geometric_map": {
        "ply_path": "data/euroc_mav/V1_01_easy/pointcloud0/data.ply",
        "vertices_bounds": {
            "min_x": min_x,
            "min_y": min_y,
            "min_z": min_z,
            "max_x": max_x,
            "max_y": max_y,
            "max_z": max_z,
        },
        "voxel_size": (0.05, 0.5),
    },
    "visual_odometry": {
        "images_directory": "data/euroc_mav/V1_01_easy/cam0/data",
        "images_ending": "png",
        "camera": {
            "width": width,
            "height": height,
            "parameters": {
                "fx": fx,
                "fy": fy,
                "cx": cx,
                "cy": cy,
                "k1": k1,
                "k2": k2,
                "p1": p1,
                "p2": p2,
            },
        },
        "initial_keyframe_index_1": initial_keyframe_index_1,
        "initial_keyframe_index_2": initial_keyframe_index_2,
        "initial_pose_1": {
            "qw": initial_poses[0][0][0],
            "qx": initial_poses[0][0][1],
            "qy": initial_poses[0][0][2],
            "qz": initial_poses[0][0][3],
            "x": initial_poses[0][1][0],
            "y": initial_poses[0][1][1],
            "z": initial_poses[0][1][2],
        },
        "initial_pose_2": {
            "qw": initial_poses[1][0][0],
            "qx": initial_poses[1][0][1],
            "qy": initial_poses[1][0][2],
            "qz": initial_poses[1][0][3],
            "x": initial_poses[1][1][0],
            "y": initial_poses[1][1][1],
            "z": initial_poses[1][1][2],
        },
        "initialize": {
            "keypoints": {
                "max_num": 760,
                "quality_level": 0.012,
                "min_distance_between": 9,
                "edge_margin": 20,
                "descriptor_angle_patch_radius": 15,
            },
            "matching": {
                "max_distance": 80,
                "max_second_to_first_distance_ratio": 1.18,
                "ransac": {
                    "threshold": 5e-5,
                    "min_num_inliers": 20,
                    "max_num_iterations": 100,
                },
            },
            "max_num_bundle_adjustment_iterations": 100,
        },
        "next": {
            "keypoints": {
                "max_num": 750,
                "quality_level": 0.011,
                "min_distance_between": 9,
                "edge_margin": 20,
                "descriptor_angle_patch_radius": 15,
            },
            "matching": {
                "max_image_distance": 23,
                "max_descriptor_distance": 84,
                "max_second_to_first_distance_ratio": 1.16,
                "ransac": {"max_num_iterations": 100, "image_distance_threshold": 3.6},
            },
            "keyframe": {
                "is_next": {
                    "min_distance_to_last": 0.3,
                    "max_num_landmark_keypoint_inliers": 60,
                    "min_num_landmark_keypoint_inliers": 18,
                },
                "max_num_keyframes": 29,
                "matching": {
                    "max_distance": 86,
                    "max_second_to_first_distance_ratio": 1.17,
                    "ransac": {
                        "threshold": 5e-5,
                        "min_num_inliers": 20,
                        "max_num_iterations": 100,
                    },
                    "max_epipolar_error": 3e-3,
                },
                "max_num_bundle_adjustment_iterations": 100,
            },
        },
    },
    "alignment": {
        "do_alignment": True,
        "align": {
            "distance_threshold_start": 2.0,
            "distance_threshold_end": 1.0,
            "max_num_alignment_steps": 10,
            "min_num_vertices_in_voxel": 10,
            "standard_deviation_scale": 3,
            "max_num_optimization_iterations": 20,
        },
    },
}


In [3]:
results_path = Path("out/align_test_refined_slides")
configs = []
raw_trajectories = []
transformed_trajectories = []

for path in results_path.iterdir():
    if (path / "raw_trajectory.csv").exists():
        raw_trajectories.append(read_trajectory_dump(path / "raw_trajectory.csv"))
        transformed_trajectories.append(
            read_trajectory_dump(path / "transformed_trajectory.csv")
        )
    else:
        continue

    with open(path / "config.json") as f:
        configs.append(json.load(f))


records = []
raw_errors = []
transformed_errors = []
for config, raw_trajectory, transformed_trajectory in zip(
    configs, raw_trajectories, transformed_trajectories
):
    vs = sorted(get_random_values(random_config, config))
    config_values = [float(v.split(": ")[-1]) for v in vs]

    raw_error = np.linalg.norm(
        (raw_trajectory - gt_trajectory[: len(raw_trajectory)]).to_numpy(), axis=1
    )
    transformed_error = np.linalg.norm(
        (
            transformed_trajectory - gt_trajectory[: len(transformed_trajectory)]
        ).to_numpy(),
        axis=1,
    )

    raw_errors.append(raw_error)
    transformed_errors.append(transformed_error)

    records.append(
        (
            raw_error.mean(),
            raw_error.std(),
            np.median(raw_error),
            raw_error.min(),
            raw_error.max(),
            *config_values,
        )
    )

raw_errors = np.array(raw_errors)
transformed_errors = np.array(transformed_errors)

config_keys = [
    "config." + v.split(": ")[0]
    for v in sorted(get_random_values(random_config, configs[0]))
]

df = pd.DataFrame(
    records,
    columns=["ate_mean", "ate_std", "ate_median", "ate_min", "ate_max", *config_keys],
)


In [4]:
fig = go.Figure()
fig.update_scenes(aspectmode="data")
fig.update_layout(margin=dict(t=10, b=0, r=0, l=0), height=800)
fig.add_scatter3d(
    x=gt_trajectory[:, 0],
    y=gt_trajectory[:, 1],
    z=gt_trajectory[:, 2],
    marker_size=2,
    name="gt",
    line_color="green",
)


for i in df.index[df.ate_mean.argsort()][[-19]]:
    fig.add_scatter3d(
        x=raw_trajectories[i].iloc[:, 0],
        y=raw_trajectories[i].iloc[:, 1],
        z=raw_trajectories[i].iloc[:, 2],
        marker_size=1.5,
        line_width=1,
        name="raw",
        line_color="red",
    )
    fig.add_scatter3d(
        x=transformed_trajectories[i].iloc[:, 0],
        y=transformed_trajectories[i].iloc[:, 1],
        z=transformed_trajectories[i].iloc[:, 2],
        marker_size=1.5,
        line_width=1,
        name="transformed",
        line_color="blue",
    )

fig


In [5]:
fig = go.Figure(layout=dict(height=600))
for i in df.index[df.ate_max.argsort()[:10]]:
    fig.add_scatter(y=raw_errors[i], line_color="red")
    fig.add_scatter(y=transformed_errors[i], line_color="blue")

fig


In [6]:
out_path = Path("out/align_manual_test/1")

raw_trajectory = read_trajectory_dump(out_path / "raw_trajectory.csv")
transformed_trajectory = read_trajectory_dump(out_path / "transformed_trajectory.csv")

fig = go.Figure()
fig.update_scenes(aspectmode="data")
fig.update_layout(
    margin=dict(t=0, b=0, r=0, l=0),
)

fig.add_scatter3d(
    x=gt_trajectory[:, 0],
    y=gt_trajectory[:, 1],
    z=gt_trajectory[:, 2],
    marker_size=1,
    name="gt",
    line_color="green",
)
fig.add_scatter3d(
    x=raw_trajectory.iloc[:, 0],
    y=raw_trajectory.iloc[:, 1],
    z=raw_trajectory.iloc[:, 2],
    marker_size=1,
    name="raw",
    line_color="red",
)
fig.add_scatter3d(
    x=transformed_trajectory.iloc[:, 0],
    y=transformed_trajectory.iloc[:, 1],
    z=transformed_trajectory.iloc[:, 2],
    marker_size=1,
    name="transformed",
    line_color="blue",
)

fig
